Real estate price prediction
Predict the real estate price of unit area of the house (imobiliária)

Kaggle Link: https://www.kaggle.com/datasets/quantbruce/real-estate-price-prediction?resource=download
UCI: https://archive.ics.uci.edu/ml/datasets/Real+estate+valuation+data+set

In [6]:
import pandas as pd
import plotly.express as px
import numpy as np
import cut_the_tails as ct
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

In [7]:
df = pd.read_csv('../data_sets\\Real_estate.csv')
target = 'Y house price of unit area'

In [8]:
features = ['X1 transaction date',
 'X2 house age',
 'X3 distance to the nearest MRT station',
 'X4 number of convenience stores',
 'X5 latitude',
 'X6 longitude']

In [9]:
fig = px.histogram(df, x=target, nbins=40)
fig.show()

# Cauda Ótima

In [12]:
df.dropna(inplace=True)
df = df.reset_index(drop=True)

model = RandomForestRegressor(max_depth=5, random_state=0) 
classifier = RandomForestClassifier(max_depth=5, random_state=0)

x,fval = ct.get_cuts_direct_optimization(df, target, features, classifier, model, 'brute')

print("Final Result: ")
print(x, fval)

[0. 0.] 0.12150784592330963
[0.   0.05] 0.12349537181979496
[0.  0.1] 0.11203886321988514
[0.   0.15] 0.12341580636031808
[0.  0.2] 0.12063067750626565
[0.   0.25] 0.11642910342694389
[0.  0.3] 0.11073607777876504
[0.   0.35] 0.11352249168128346
[0.  0.4] 0.12878448527729666
[0.   0.45] 0.13447093816003497
[0.  0.5] 0.1442509730396705
[0.   0.55] 0.14674293777804628
[0.  0.6] 0.13200542379047367
[0.   0.65] 0.1388715978669629
[0.  0.7] 0.13383113358556567
[0.   0.75] 0.1333519717449526
[0.  0.8] 0.13378344679124116
[0.   0.85] 0.1394579955444812
[0.  0.9] 0.1283048399216805
[0.   0.95] 0.12713804868308817
[0.   0.05] 0.12349537181979496
[0.05 0.05] 0.12326364675500967
[0.05 0.1 ] 0.11927624043792413
[0.05 0.15] 0.1297139652185626
[0.05 0.2 ] 0.13101414536884218
[0.05 0.25] 0.13016287301016932
[0.05 0.3 ] 0.12731628928690844
[0.05 0.35] 0.12778240560885165
[0.05 0.4 ] 0.14717696934523597
[0.05 0.45] 0.14636848894292234
[0.05 0.5 ] 0.15568811824098225
[0.05 0.55] 0.1572677006009921
[0.05

In [29]:
cdf = ct.split_by_quantile_class(df,target,[0.17,0.5])

In [30]:
X = cdf[features].to_numpy()
y_tail = cdf['tail_class'].to_numpy()
y = cdf[target].to_numpy()

###Using split###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train_aux, X_test_aux, y_train_tail, y_test_tail = train_test_split(X, y_tail, test_size=0.2, random_state=0)

In [31]:
#baseline = RandomForestRegressor(max_depth=5, random_state=0)
baseline = XGBRegressor(n_estimators=100, max_leaves=0, random_state = 0)
#baseline = DecisionTreeRegressor(random_state=0, max_depth=5) 

In [32]:
#tail_classifier = ct.fit_tail_classifier(X,y_tail,RandomForestClassifier(max_depth=5, random_state=0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X,y_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 

###Using split###
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,RandomForestClassifier(max_depth=5, random_state=0))
tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,XGBClassifier(n_estimators=100, max_leaves=0, random_state = 0))
#tail_classifier = ct.fit_tail_classifier(X_train,y_train_tail,DecisionTreeClassifier(random_state=0, max_depth=5)) 

In [33]:
#models = ct.fit_tail_models(X,y,y_tail,baseline)

###Using split###
models = ct.fit_tail_models(X_train,y_train,y_train_tail,baseline)

In [34]:
#baseline.fit(X,y)
#y_base = baseline.predict(X)
#y_tail = ct.batch_tail_predict(X,tail_classifier,models)

###Using split###
baseline.fit(X_train,y_train)
y_base = baseline.predict(X_test)
y_tail = ct.batch_tail_predict(X_test,tail_classifier,models)

In [35]:
#print(mean_absolute_error(y_base,y))
#print(mean_absolute_error(y_tail,y))
#print(mean_absolute_error(y,y))

###Using split###
print(mean_absolute_error(y_base,y_test))
print(mean_absolute_error(y_tail,y_test))
print(mean_absolute_error(y_test,y_test))

5.899335498120411
5.557011257309511
0.0


In [36]:
#print(mean_absolute_percentage_error(y_base,y))
#print(mean_absolute_percentage_error(y_tail,y))
#print(mean_absolute_percentage_error(y,y))

###Using split###
print(mean_absolute_percentage_error(y_base,y_test))
print(mean_absolute_percentage_error(y_tail,y_test))
print(mean_absolute_percentage_error(y_test,y_test))

0.19269933830018918
0.15102807249860936
0.0
